In [2]:
#setup bedrock
import json
import os
import sys

import boto3
import botocore
import bedrock  

module_path = ".."
sys.path.append(os.path.abspath(module_path))

boto3_bedrock = bedrock.get_bedrock_client(
    assumed_role=os.environ.get("BEDROCK_ASSUME_ROLE", None),
    region=os.environ.get("AWS_DEFAULT_REGION", None),
    runtime=False
)

Create new client
  Using region: us-east-1
boto3 Bedrock client successfully created!
bedrock(https://bedrock.us-east-1.amazonaws.com)


In [3]:
boto3_bedrock.list_foundation_models()

{'ResponseMetadata': {'RequestId': '1b81644c-d665-4adc-b2d4-82ae35336285',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Fri, 01 Mar 2024 17:17:42 GMT',
   'content-type': 'application/json',
   'content-length': '16744',
   'connection': 'keep-alive',
   'x-amzn-requestid': '1b81644c-d665-4adc-b2d4-82ae35336285'},
  'RetryAttempts': 0},
 'modelSummaries': [{'modelArn': 'arn:aws:bedrock:us-east-1::foundation-model/amazon.titan-tg1-large',
   'modelId': 'amazon.titan-tg1-large',
   'modelName': 'Titan Text Large',
   'providerName': 'Amazon',
   'inputModalities': ['TEXT'],
   'outputModalities': ['TEXT'],
   'responseStreamingSupported': True,
   'customizationsSupported': [],
   'inferenceTypesSupported': ['ON_DEMAND'],
   'modelLifecycle': {'status': 'ACTIVE'}},
  {'modelArn': 'arn:aws:bedrock:us-east-1::foundation-model/amazon.titan-image-generator-v1:0',
   'modelId': 'amazon.titan-image-generator-v1:0',
   'modelName': 'Titan Image Generator G1',
   'providerName': 'Amazon',

In [4]:
import os
from langchain.embeddings import BedrockEmbeddings
from langchain.indexes import VectorstoreIndexCreator
from langchain.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import PyPDFLoader
from langchain.llms.bedrock import Bedrock
from langchain.text_splitter import CharacterTextSplitter

In [5]:
def create_llm(model_name):
    model_kwargs =  { 
        "maxTokenCount": 1024, 
        "stopSequences": [], 
        "temperature": 0, 
        "topP": 0.9 }
    llm = Bedrock(
        credentials_profile_name=os.environ.get("PROFILE"), 
        region_name=os.environ.get("REGION"), 
        endpoint_url=os.environ.get("ENDPOINT"), 
        model_id=model_name,
        model_kwargs=model_kwargs) 
    return llm

In [27]:
def create_get_index_new(): 
    embeddings = BedrockEmbeddings(
        credentials_profile_name=os.environ.get("PROFILE"), 
        region_name=os.environ.get("REGION"), 
        endpoint_url=os.environ.get("ENDPOINT"),) 
    pdf_path = "chinookSchema_2.pdf" 
    loader = PyPDFLoader(file_path=pdf_path) 
    text_splitter = CharacterTextSplitter(separator=";",
                                          chunk_size=1000,
                                          chunk_overlap=100,
                                          length_function=len,
                                          is_separator_regex=False,)

    index_creator = VectorstoreIndexCreator( 
        vectorstore_cls=FAISS, 
        embedding=embeddings, 
        text_splitter=text_splitter, )
    index_from_loader = index_creator.from_loaders([loader]) 
    return index_from_loader 

In [28]:
def call_rag_function(index, input_text): 
    model_name="amazon.titan-tg1-large"
    llm = create_llm(model_name)
    response_text = index.query(question=input_text, llm=llm) 
    return response_text

In [29]:
create_get_index_new()

VectorStoreIndexWrapper(vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x7fcb6c1aa530>)

In [30]:
index = create_get_index_new()

In [34]:
text = "Get number of items in album table"

In [35]:
response_content = call_rag_function(index=index, input_text=text)

In [36]:
response_content

'SELECT count(*) FROM album\n'